# Some Tree Data!
Note: GitHub does not render Folium maps, so to view this properly visit [here](https://nbviewer.jupyter.org/github/CalLanherne/Tree-Data-in-Bristol/blob/master/tree_plantings_bristol.ipynb)


This is mini project to practise manipulating geospatial datasets and displaying them on graphs. Using Bristol City Council's open data, we find the boundaries of wards, and the location of new tree plantings. Then we find out which ward each new tree is in, and plot this using a Choropleth map. We also plot each individual tree using cluster markers, with the label being the common name of the tree.

In [2]:
import requests
import folium
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
from shapely.geometry import Point,Polygon
import geopandas as gpd
from folium import plugins
import matplotlib.pyplot as plt
%matplotlib inline

This function makes a dictionary formatted in the geojson standard (https://geojson.org/) for use by `Folium`

In [3]:
def extract_geojson(ward_fields):
    geo_dict = {'type':'Feature'}
    ward_name = ward_fields['name']
    geom_type = ward_fields['geo_shape']['type']
    geom_coords = ward_fields['geo_shape']['coordinates']
    geo_dict['geometry']={'type':geom_type,'coordinates':geom_coords}
    geo_dict['properties']={'name':ward_name}
    return geo_dict

Fetch data from Bristol OpenData and check that this has been done correctly

In [4]:
bristol_data_url = 'https://opendata.bristol.gov.uk/api/records/1.0/search/?dataset='
wards_id = 'wards'
tree_planting_id = 'tree-planting-locations'
ward_resp = requests.get(bristol_data_url+wards_id+'&q=&rows=-1')
if ward_resp.status_code != 200:
    raise ApiError('GET {}'.format(ward_resp.status_code))   
    
tree_planting_resp =  requests.get(bristol_data_url+tree_planting_id+'&q=&rows=-1')
if tree_planting_resp.status_code != 200:
    raise ApiError('GET {}'.format(tree_planting_resp.status_code))
    

In [5]:
wards_db = ward_resp.json()['records']

In [6]:
wards_geojson = {'type':'FeatureCollection','features':[extract_geojson(wards_db[i]['fields']) for i in range(len(wards_db))]}

#with open('wards.json','w') as json_file:
#    json.dump(wards_geojson,json_file,indent=2)

Flatten data structures and tidy up unnecessary/duplicate features

In [7]:
wards_db_norm = pd.json_normalize(wards_db)
wards_db_norm.drop(columns=['datasetid','recordid','record_timestamp',
             'fields.objectid','fields.geo_point_2d','fields.ward_id',
             'fields.councillors','fields.geo_shape.type','geometry.type',
             'geometry.coordinates'],inplace=True)
wards_db_norm.rename(columns = {'fields.name':'ward_name','fields.geo_shape.coordinates':'ward_boundary'},inplace=True)
wards_db_norm['polygon'] = [Polygon(vertices[0]) for vertices in wards_db_norm['ward_boundary']]

In [8]:
tree_db = pd.json_normalize(tree_planting_resp.json()['records'])
tree_db.drop(columns=['datasetid','recordid','record_timestamp','fields.asset_id',
             'fields.objectid','fields.latin_code','fields.site_code',
             'fields.y','fields.plot_number','fields.x','fields.sponsorship',
             'fields.feature_type_name','geometry.coordinates','geometry.type',
             'fields.geo_shape.type','fields.geo_shape.coordinates'],inplace=True)
tree_db.rename(columns = {'fields.site_name':'location','fields.latin_name':'latin_name',
                          'fields.full_common_name':'full_common_name','fields.common_name':'common_name',
                          'fields.feature_start_date':'start_date','fields.geo_point_2d':'coordinates'},inplace=True)
tree_db['start_date']=pd.to_datetime(tree_db['start_date']).dt.date

Some common names are missing - replace these with the latin names

In [9]:
tree_db['full_common_name'][tree_db['full_common_name'].isnull()] = tree_db['latin_name'][tree_db['full_common_name'].isnull()]

Function to check if tree is in ward using `shapely` polygons

In [10]:
#Inefficient but easy to understand!
def assign_ward_to_tree(coord):
    tree_point = Point(coord[1],coord[0])
    
    for ward,ward_polygon in zip(wards_db_norm['ward_name'],wards_db_norm['polygon']):        
        if tree_point.within(ward_polygon):
            return ward
    return 'Not in ward'
    

In [11]:
tree_db['ward'] = [assign_ward_to_tree(coord) for coord in tree_db['coordinates']]

In [12]:
tree_group = pd.DataFrame(tree_db.groupby('ward').size())
tree_group.reset_index(inplace=True)
tree_group.columns = ['Wards', 'Count']
tree_group.sort_values(['Count'],ascending=False).head()

,Wards,Count
12,Hartcliffe & Withywood,132
1,Avonmouth & Lawrence Weston,125
13,Henbury & Brentry,69
28,Westbury-on-Trym & Henleaze,64
16,Horfield,55


In [13]:
bristol_coordinates = [51.4545,-2.5879]
bristol_map = folium.Map(location = bristol_coordinates,zoom_start=12,tiles='Stamen Toner')
trees = plugins.MarkerCluster().add_to(bristol_map)
for coord, label in zip(tree_db['coordinates'],tree_db['full_common_name']):
    folium.Marker(
        location=[coord[0], coord[1]],
        icon=None,
        popup=label,
    ).add_to(trees)
folium.Choropleth(geo_data=wards_geojson,
                  data=tree_group,
                  columns=['Wards','Count'],
                  key_on = 'feature.properties.name',
                  fill_color = 'BuGn',
                  fill_opacity = 0.7,
                  line_opacity = 0.2,
                  legend_name='New tree plantings since {}'.format(tree_db['start_date'].min())).add_to(bristol_map)
bristol_map

In [14]:
bristol_map.save('BristolTreeMap.html')